# Imports

In [31]:
import os
import json
import time
import pandas as pd
import numpy as np
import opensim as osim
import matplotlib.pyplot as plt
import msk_modelling_python as msk
import matplotlib.pyplot as plt
parent_dir = os.path.dirname(os.getcwd())
start_time = time.time()
################

import vtk


# Test functions

In [2]:
x = 10
x -= 1

print(x)

9


# Build knee model

## Create functions

## Create bodies and joints

In [32]:

# Define global model where the leg lives.
leg = osim.Model()
leg.setUseVisualizer(True)

# ---------------------------------------------------------------------------
# Create two links, each with a mass of 1 kg, centre of mass at the body's
# origin, and moments and products of inertia of zero.
# ---------------------------------------------------------------------------

length_femur = 0.5
length_tibia = 1
length_foot = 0.5

femur = osim.Body("femur",
                    length_femur,
                    osim.Vec3(0, 0, 0),
                    osim.Inertia(0, 0, 0))
tibia = osim.Body("tibia",
                   length_tibia,
                   osim.Vec3(0, 0, 0),
                   osim.Inertia(0, 0, 0))

foot = osim.Body("foot",
                    length_foot,
                    osim.Vec3(0, 0, 0),
                    osim.Inertia(0, 0, 0))


# ---------------------------------------------------------------------------
# Connect the bodies with pin joints. Assume each body is 1m long.
# ---------------------------------------------------------------------------

hip = osim.PinJoint("hip",
                    leg.getGround(),  # PhysicalFrame
                    osim.Vec3(0, 0, 0),
                    osim.Vec3(0, 0, 0),
                    femur,  # PhysicalFrame
                    osim.Vec3(0, length_femur, 0),
                    osim.Vec3(0, 0, 0))

knee = osim.PinJoint("knee",
                     femur,  # PhysicalFrame
                     osim.Vec3(0, 0, 0),
                     osim.Vec3(0, 0, 0),
                     tibia,  # PhysicalFrame
                     osim.Vec3(0, length_tibia, 0),
                     osim.Vec3(0, 0, 0))

ankle = osim.PinJoint("ankle",
                        tibia,  # PhysicalFrame
                        osim.Vec3(0, 0, 0),
                        osim.Vec3(0, 0, 0),
                        foot,  # PhysicalFrame
                        osim.Vec3(0, 0.5, 0),
                        osim.Vec3(0, 0, 0))


## Create muscles and controller (brain)

In [33]:

# ---------------------------------------------------------------------------
# Add a muscle that flexes the elbow (actuator for robotics people).
# ---------------------------------------------------------------------------
quadriceps = osim.Millard2012EquilibriumMuscle("quadriceps",  # Muscle name
                                               300.0,  # Max isometric force
                                               0.6,  # Optimal fibre length
                                               0.55,  # Tendon slack length
                                               0.0)  # Pennation angle
quadriceps.addNewPathPoint("origin",
                           femur,
                           osim.Vec3(0, 0.8, 0))

quadriceps.addNewPathPoint("insertion",
                           tibia,
                           osim.Vec3(0, 0.7, 0))

hamstrings = osim.Millard2012EquilibriumMuscle("hamstrings",  # Muscle name
                                               200.0,  # Max isometric force
                                               0.6,  # Optimal fibre length
                                               0.55,  # Tendon slack length
                                               0.0)  # Pennation angle
hamstrings.addNewPathPoint("origin",
                           femur,
                           osim.Vec3(0, 0.2, 0))

hamstrings.addNewPathPoint("insertion",
                           tibia,
                           osim.Vec3(0, 1, 0))

# ---------------------------------------------------------------------------
# Add a controller that specifies the excitation of the muscles.
# ---------------------------------------------------------------------------

brain = osim.PrescribedController()
brain.addActuator(quadriceps)
brain.addActuator(hamstrings)
brain.prescribeControlForActuator("quadriceps",
                                  osim.StepFunction(0.1, 0.2, 0.3, 0.3))
brain.prescribeControlForActuator("hamstrings",
                                  osim.StepFunction(0.5, 1.0, 2.3, 3.0))



## Add components to the model

In [34]:
# ---------------------------------------------------------------------------
# Build model with components created above.
# ---------------------------------------------------------------------------

leg.addBody(femur)
leg.addBody(tibia)
leg.addBody(foot)
leg.addJoint(hip)
leg.addJoint(knee)
leg.addJoint(ankle)
leg.addForce(quadriceps)
leg.addForce(hamstrings)
leg.addController(brain)

# ---------------------------------------------------------------------------
# Add a console reporter to print the muscle fibre force and knee angle.
# ---------------------------------------------------------------------------

# We want to write our simulation results to the console.
reporter = osim.ConsoleReporter()
reporter.set_report_time_interval(1.0)
reporter.addToReport(quadriceps.getOutput("fiber_force"))
reporter.addToReport(hamstrings.getOutput("fiber_force"))
knee_coord = knee.getCoordinate().getOutput("value")
reporter.addToReport(knee_coord, "knee_angle")
leg.addComponent(reporter)

# ---------------------------------------------------------------------------
# Add display geometry. 
# ---------------------------------------------------------------------------

bodyGeometry = osim.Ellipsoid(0.1, 0.5, 0.1)
bodyGeometry.setColor(osim.Gray)
femurCenter = osim.PhysicalOffsetFrame()
femurCenter.setName("femurCenter")
femurCenter.setParentFrame(femur)
femurCenter.setOffsetTransform(osim.Transform(osim.Vec3(0, 0.5, 0)))
femur.addComponent(femurCenter)
femurCenter.attachGeometry(bodyGeometry.clone())

tibiaCenter = osim.PhysicalOffsetFrame()
tibiaCenter.setName("tibiaCenter")
tibiaCenter.setParentFrame(tibia)
tibiaCenter.setOffsetTransform(osim.Transform(osim.Vec3(0, 0.5, 0)))
tibia.addComponent(tibiaCenter)
tibiaCenter.attachGeometry(bodyGeometry.clone())

foot_vtp = r"C:\Users\Bas\Downloads\Final 3D STL Models-stl\Left\VHM_Left_Bone_Phalanges_smooth.stl"
ankleBodyGeometry = osim.Mesh(foot_vtp)
ankleCenter = osim.PhysicalOffsetFrame()
ankleCenter.setName("ankleCenter")
ankleCenter.setParentFrame(foot)
ankleCenter.setOffsetTransform(osim.Transform(osim.Vec3(0, 0.25, 0)))
foot.addComponent(ankleCenter)
ankleCenter.attachGeometry(bodyGeometry.clone())




# Build knee model

## Create bodies and joints

In [27]:

# Define global model where the leg lives.
leg = osim.Model()
leg.setUseVisualizer(True)

# ---------------------------------------------------------------------------
# Create two links, each with a mass of 1 kg, centre of mass at the body's
# origin, and moments and products of inertia of zero.
# ---------------------------------------------------------------------------

length_femur = 0.5
length_tibia = 1
length_foot = 0.5

femur = osim.Body("femur",
                    length_femur,
                    osim.Vec3(0, 0, 0),
                    osim.Inertia(0, 0, 0))
tibia = osim.Body("tibia",
                   length_tibia,
                   osim.Vec3(0, 0, 0),
                   osim.Inertia(0, 0, 0))

foot = osim.Body("foot",
                    length_foot,
                    osim.Vec3(0, 0, 0),
                    osim.Inertia(0, 0, 0))


# ---------------------------------------------------------------------------
# Connect the bodies with pin joints. Assume each body is 1m long.
# ---------------------------------------------------------------------------

hip = osim.PinJoint("hip",
                    leg.getGround(),  # PhysicalFrame
                    osim.Vec3(0, 0, 0),
                    osim.Vec3(0, 0, 0),
                    femur,  # PhysicalFrame
                    osim.Vec3(0, length_femur, 0),
                    osim.Vec3(0, 0, 0))

knee = osim.PinJoint("knee",
                     femur,  # PhysicalFrame
                     osim.Vec3(0, 0, 0),
                     osim.Vec3(0, 0, 0),
                     tibia,  # PhysicalFrame
                     osim.Vec3(0, length_tibia, 0),
                     osim.Vec3(0, 0, 0))

ankle = osim.PinJoint("ankle",
                        tibia,  # PhysicalFrame
                        osim.Vec3(0, 0, 0),
                        osim.Vec3(0, 0, 0),
                        foot,  # PhysicalFrame
                        osim.Vec3(0, 0.5, 0),
                        osim.Vec3(0, 0, 0))


## Create muscles and controller (brain)

In [28]:

# ---------------------------------------------------------------------------
# Add a muscle that flexes the elbow (actuator for robotics people).
# ---------------------------------------------------------------------------
quadriceps = osim.Millard2012EquilibriumMuscle("quadriceps",  # Muscle name
                                               300.0,  # Max isometric force
                                               0.6,  # Optimal fibre length
                                               0.55,  # Tendon slack length
                                               0.0)  # Pennation angle
quadriceps.addNewPathPoint("origin",
                           femur,
                           osim.Vec3(0, 0.8, 0))

quadriceps.addNewPathPoint("insertion",
                           tibia,
                           osim.Vec3(0, 0.7, 0))

hamstrings = osim.Millard2012EquilibriumMuscle("hamstrings",  # Muscle name
                                               200.0,  # Max isometric force
                                               0.6,  # Optimal fibre length
                                               0.55,  # Tendon slack length
                                               0.0)  # Pennation angle
hamstrings.addNewPathPoint("origin",
                           femur,
                           osim.Vec3(0, 0.2, 0))

hamstrings.addNewPathPoint("insertion",
                           tibia,
                           osim.Vec3(0, 1, 0))

# ---------------------------------------------------------------------------
# Add a controller that specifies the excitation of the muscles.
# ---------------------------------------------------------------------------

brain = osim.PrescribedController()
brain.addActuator(quadriceps)
brain.addActuator(hamstrings)
brain.prescribeControlForActuator("quadriceps",
                                  osim.StepFunction(0.1, 0.2, 0.3, 0.3))
brain.prescribeControlForActuator("hamstrings",
                                  osim.StepFunction(0.5, 1.0, 2.3, 3.0))



## Add components to the model

In [29]:
# ---------------------------------------------------------------------------
# Build model with components created above.
# ---------------------------------------------------------------------------

leg.addBody(femur)
leg.addBody(tibia)
leg.addBody(foot)
leg.addJoint(hip)
leg.addJoint(knee)
leg.addJoint(ankle)
leg.addForce(quadriceps)
leg.addForce(hamstrings)
leg.addController(brain)

# ---------------------------------------------------------------------------
# Add a console reporter to print the muscle fibre force and knee angle.
# ---------------------------------------------------------------------------

# We want to write our simulation results to the console.
reporter = osim.ConsoleReporter()
reporter.set_report_time_interval(1.0)
reporter.addToReport(quadriceps.getOutput("fiber_force"))
reporter.addToReport(hamstrings.getOutput("fiber_force"))
knee_coord = knee.getCoordinate().getOutput("value")
reporter.addToReport(knee_coord, "knee_angle")
leg.addComponent(reporter)

# ---------------------------------------------------------------------------
# Add display geometry. 
# ---------------------------------------------------------------------------

bodyGeometry = osim.Ellipsoid(0.1, 0.5, 0.1)
bodyGeometry.setColor(osim.Gray)
femurCenter = osim.PhysicalOffsetFrame()
femurCenter.setName("femurCenter")
femurCenter.setParentFrame(femur)
femurCenter.setOffsetTransform(osim.Transform(osim.Vec3(0, 0.5, 0)))
femur.addComponent(femurCenter)
femurCenter.attachGeometry(bodyGeometry.clone())

tibiaCenter = osim.PhysicalOffsetFrame()
tibiaCenter.setName("tibiaCenter")
tibiaCenter.setParentFrame(tibia)
tibiaCenter.setOffsetTransform(osim.Transform(osim.Vec3(0, 0.5, 0)))
tibia.addComponent(tibiaCenter)
tibiaCenter.attachGeometry(bodyGeometry.clone())

foot_vtp = r"C:\Users\Bas\Downloads\Final 3D STL Models-stl\Left\VHM_Left_Bone_Phalanges_smooth.stl"
ankleBodyGeometry = osim.Mesh(foot_vtp)
ankleCenter = osim.PhysicalOffsetFrame()
ankleCenter.setName("ankleCenter")
ankleCenter.setParentFrame(foot)
ankleCenter.setOffsetTransform(osim.Transform(osim.Vec3(0, 0.25, 0)))
foot.addComponent(ankleCenter)
ankleCenter.attachGeometry(bodyGeometry.clone())




# Initiate the system and run the model simulation

In [35]:
# ---------------------------------------------------------------------------
# Configure the model.
# ---------------------------------------------------------------------------

state = leg.initSystem()
# Fix the hip at its default angle and begin with the knee flexed.
hip.getCoordinate().setLocked(state, True)
knee.getCoordinate().setValue(state, osim.SimTK_PI) # 180 degrees
ankle.getCoordinate().setValue(state, 0.5) # 90 degrees
leg.equilibrateMuscles(state)

# ---------------------------------------------------------------------------
# Simulate.
# ---------------------------------------------------------------------------

manager = osim.Manager(leg)
state.setTime(0)
manager.initialize(state)
state = manager.integrate(10.0)

# ---------------------------------------------------------------------------
# Print/save model file
# ---------------------------------------------------------------------------

leg.printToXML(os.path.join(parent_dir, "leg.osim"))



True

# End test

In [16]:
################
print(f"--- {time.time() - start_time} seconds ---")

--- 3.2267627716064453 seconds ---
